In [1]:
import os
import sys
import pandas as pd
import numpy as np
import requests
import re
import io
from datetime import datetime
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from konlpy.tag import Okt 
import time
import json
import csv

In [16]:
#톡톡-대한민국이슈 실시간 베스트글 실시간 (당일 기준 게시물 최대 10개가 추려지는 게시판)
def nate_Crawler():
    
    browser = Chrome()
    browser.maximize_window()
    base_url = 'https://pann.nate.com'
    browser.get(base_url)
    browser.find_elements_by_xpath('//*[@id="header"]/div[2]/div/ul/li[2]/a')[0].click()
    browser.find_elements_by_xpath('//*[@id="categoryArea"]/li[13]/a')[0].click()
    
    titles = []
    links = []
    link = []

    res = requests.get(browser.current_url)
    res.raise_for_status()
    res.encoding = None
    html = res.text

    soup = BeautifulSoup(html,'html.parser')
    full_list = soup.find('ul', {'class': 'post_list'}).findAll('li')

    for li in full_list: 
        link = li.findAll('a')
        for a in link:
            if "channel" in a['href']:
                continue
            else:
                real_link = 'https://pann.nate.com' + a.get('href')  
                real_title = a.get('title')

        links.append(real_link)
        titles.append(real_title)


    txt = []
    for i in links:
        res = requests.get(i)
        res.raise_for_status()
        res.encoding = None
        html2 = res.text

        soup = BeautifulSoup(html2, 'html.parser')
        contentArea = soup.find("div", {"class" : "viewarea"})
        parags = contentArea.findAll("div", {"id" : "contentArea"})

        content = ""

        for parag in parags:
            content += parag.text.replace('&nbsp;| |\t|\r|\n|', '')
        content = content.replace('\n',"")
        content = content.replace('\t',"")
        content = content.replace('\xa0',"")
        content = content.replace('\u200b',"")
        content = content.replace('\ufeff',"")
        content = content.replace('[',"")
        content = content.replace(']',"")
        content = content.replace('..|...|....',"")
        content = content.replace('ㅋㅋㅋㅋ|ㅋㅋㅋㅋㅋㅋㅋ|ㅋㅋㅋㅋㅋㅋㅋㅋ|ㅎㅎㅎㅎ|ㄷㄷ',"")
        txt.append(content)

    okt=Okt()
    morphs = []
    for i in txt:
            morphs.append(okt.morphs(i))
            
    
    nate_dict = {
        '제목' : titles,
        '본문' : txt
    }

    df = pd.DataFrame(nate_dict) 
    df.to_csv('natepann.csv', index=False, encoding="utf-8-sig")
    
    morphs_dict = {
        'gud'
    }
    df2 = pd.DataFrame(morphs, header = None)
    df2.to_csv('natepann_Morphs.csv', index=False, encoding="utf-8-sig")
    
    browser.close()

In [17]:
nate_Crawler()

TypeError: __init__() got an unexpected keyword argument 'header'